In [1]:
import magellan as mg

DEBUG:Cloud:Log file (/Users/clarence/.picloud/cloud.log) opened


In [2]:
# initialize JVM
mg.init_jvm()

True

In [3]:
# read in csv files
A= mg.read_csv('tableA.csv',key='id')
B= mg.read_csv('tableB.csv',key='id')

# Read in Golden Table and Split the data

In [4]:
# read in the labeled data
G = mg.read_csv('OutputG1.csv', ltable=A, rtable=B)
len(G)

400

In [5]:
# Split G into development (I) and evaluation (J)
IJ = mg.train_test_split(G, train_proportion=0.65)
I = IJ['train']
J = IJ['test']
# check the number of tuples in I and J
(len(I), len(J))

(260, 140)

In [6]:
# output I and J in case we need to do re-label
I.to_csv('I.csv')
J.to_csv('J.csv')

True

# Plan for selecting best matcher

In [ ]:
# Selecting the best learning-based matcher using I

# Plan 

# 1. Create a set of ML-matchers
# 2. Generate features --> feature_table
# 3. Extract feature vectors using I and feature_table
# 4. Select best learning-based matcher using CV
# 5. Debug the selected matcher (and repeat the above steps)

In [7]:
# Create a set of ML-matchers
dt = mg.DTMatcher(name='DecisionTree', random_state=0)
svm = mg.SVMMatcher()
rf = mg.RFMatcher(name='RF', random_state=0)
nb = mg.NBMatcher(name='NB')
lg = mg.LogRegMatcher(name='LogReg')
ln = mg.LinRegMatcher(name='LinReg')

In [8]:
# check names of the matchers
(dt.name, rf.name, svm.name, nb.name, lg.name, ln.name)

('DecisionTree', 'RF', 'SVM_65898401806274538441', 'NB', 'LogReg', 'LinReg')

# Training Iteration 1

In [10]:
# Generate features
feat_table = mg.get_features_for_matching(A, B)
feat_table

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source
0,id_id_exm,id,id,None,None,exact_match,<function id_id_exm at 0x12376d0c8>,from magellan.feature.simfunctions import *\nf...
1,id_id_anm,id,id,None,None,abs_norm,<function id_id_anm at 0x12376d2a8>,from magellan.feature.simfunctions import *\nf...
2,id_id_lev,id,id,None,None,lev,<function id_id_lev at 0x123691848>,from magellan.feature.simfunctions import *\nf...
3,name_name_jac_qgm_3_qgm_3,name,name,qgm_3,qgm_3,jaccard,<function name_name_jac_qgm_3_qgm_3 at 0x12369...,from magellan.feature.simfunctions import *\nf...
4,name_name_cos_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,cosine,<function name_name_cos_dlm_dc0_dlm_dc0 at 0x1...,from magellan.feature.simfunctions import *\nf...
5,name_name_jac_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,jaccard,<function name_name_jac_dlm_dc0_dlm_dc0 at 0x1...,from magellan.feature.simfunctions import *\nf...
6,name_name_mel,name,name,None,None,monge_elkan,<function name_name_mel at 0x123691a28>,from magellan.feature.simfunctions import *\nf...
7,name_name_lev,name,name,None,None,lev,<function name_name_lev at 0x123691aa0>,from magellan.feature.simfunctions import *\nf...
8,name_name_nmw,name,name,None,None,needleman_wunsch,<function name_name_nmw at 0x123691b18>,from magellan.feature.simfunctions import *\nf...
9,name_name_sw,name,name,None,None,smith_waterman,<function name_name_sw at 0x123691b90>,from magellan.feature.simfunctions import *\nf...


In [11]:
# sim function, token and corres
mg._match_s
mg._match_t
mg._match_c['corres']

[('id', 'id'),
 ('name', 'name'),
 ('streetAddress', 'streetAddress'),
 ('city', 'city'),
 ('state', 'state'),
 ('zipCode', 'zipCode'),
 ('website', 'website'),
 ('priceRange', 'priceRange'),
 ('ratingValue', 'ratingValue'),
 ('neighborhood', 'neighborhood')]

In [12]:
# Select 'name', 'streetAddress', 'city', 'state', 'zipCode' related features
feat_subset_iter1 = feat_table[3:36]

In [13]:
# Get feature vectors
H = mg.extract_feature_vecs(I, feature_table=feat_subset_iter1, attrs_after='gold')
H.head()

,_id,ltable.id,rtable.id,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev,name_name_nmw,name_name_sw,...,state_state_lev,state_state_jar,state_state_jwn,state_state_sdx,state_state_exm,state_state_jac_qgm_3_qgm_3,zipCode_zipCode_exm,zipCode_zipCode_anm,zipCode_zipCode_lev,gold
0,0,6569,19993,0.454545,0.500000,0.333333,0.775000,0.650000,0.882353,0.750000,...,1,1,1,1,1,0,0,0.999834,0.428571,0
1,1,5883,15732,0.310345,0.408248,0.250000,0.725000,0.500000,0.711538,0.687500,...,1,1,1,1,1,0,0,0.999718,0.428571,0
2,2,7939,23163,0.652174,0.666667,0.500000,0.578947,0.863636,0.950000,0.973684,...,1,1,1,1,1,0,1,1.000000,0.714286,1
3,3,4812,12321,0.724138,0.750000,0.600000,0.520000,0.892857,0.961538,0.980000,...,1,1,1,1,1,0,1,1.000000,0.714286,1
4,4,9081,27265,0.250000,0.408248,0.250000,0.615385,0.476190,0.714286,0.692308,...,1,1,1,1,1,0,0,0.999853,0.428571,0


In [14]:
# impute H
H.fillna(0, inplace=True)
H.head()

,_id,ltable.id,rtable.id,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev,name_name_nmw,name_name_sw,...,state_state_lev,state_state_jar,state_state_jwn,state_state_sdx,state_state_exm,state_state_jac_qgm_3_qgm_3,zipCode_zipCode_exm,zipCode_zipCode_anm,zipCode_zipCode_lev,gold
0,0,6569,19993,0.454545,0.500000,0.333333,0.775000,0.650000,0.882353,0.750000,...,1,1,1,1,1,0,0,0.999834,0.428571,0
1,1,5883,15732,0.310345,0.408248,0.250000,0.725000,0.500000,0.711538,0.687500,...,1,1,1,1,1,0,0,0.999718,0.428571,0
2,2,7939,23163,0.652174,0.666667,0.500000,0.578947,0.863636,0.950000,0.973684,...,1,1,1,1,1,0,1,1.000000,0.714286,1
3,3,4812,12321,0.724138,0.750000,0.600000,0.520000,0.892857,0.961538,0.980000,...,1,1,1,1,1,0,1,1.000000,0.714286,1
4,4,9081,27265,0.250000,0.408248,0.250000,0.615385,0.476190,0.714286,0.692308,...,1,1,1,1,1,0,0,0.999853,0.428571,0


In [15]:
# select the best ML matcher using CV
result_p = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='precision', random_state=0)
result_r = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='recall', random_state=0)
result_f1 = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='f1', random_state=0)
# print results
print "Precision: "
print result_p['cv_stats']
print "Recall: "
print result_r['cv_stats']
print "F1: "
print result_f1['cv_stats']

Precision: 
                       Name  \
0              DecisionTree   
1                        RF   
2  SVM_65898401806274538441   
3                        NB   
4                    LogReg   
5                    LinReg   

                                             Matcher  Num folds    Fold 1  \
0  <magellan.matcher.dtmatcher.DTMatcher object a...          5  1.000000   
1  <magellan.matcher.rfmatcher.RFMatcher object a...          5  0.916667   
2  <magellan.matcher.svmmatcher.SVMMatcher object...          5  1.000000   
3  <magellan.matcher.nbmatcher.NBMatcher object a...          5  0.800000   
4  <magellan.matcher.logregmatcher.LogRegMatcher ...          5  1.000000   
5  <magellan.matcher.linregmatcher.LinRegMatcher ...          5  1.000000   

     Fold 2    Fold 3    Fold 4    Fold 5  Mean score  
0  0.857143  1.000000  1.000000  0.750000    0.921429  
1  1.000000  1.000000  1.000000  1.000000    0.983333  
2  1.000000  1.000000  1.000000  1.000000    1.000000  
3  0.8

In [ ]:
# Selecting the best ML matcher:
# consider Precision as high as possible
# break tie using recall

In [ ]:
# select SVM as the best matcher

In [16]:
# Split feature vectors to train and test
UV = mg.train_test_split(H, train_proportion=0.5, random_state=0)
U = UV['train']
V = UV['test']

In [ ]:
# use DT and RF for proxy debugging

In [17]:
# debug DT results
mg.vis_debug_dt(dt, U, V, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')
# precision: 95.45% (21/22)
# recall: 84.0% (21/25)
# F1: 89.36%

In [18]:
# debug RF results
mg.vis_debug_rf(rf, U, V, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')
# precision: 95.83% (23/24)
# recall: 92.0% (23/25)
# F1: 93.88%

In [19]:
# check SVM results
svm.fit(table=U, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')
# Predict V using SVM
P_svm = svm.predict(table=V, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)
# Evaluate the predictions
eval_result = mg.eval_matches(P_svm, 'gold', 'predicted')
mg.print_eval_summary(eval_result)
# Precision : 100.0% (19/19)
# Recall : 76.0% (19/25)
# F1 : 86.36%

Precision : 100.0% (19/19)
Recall : 76.0% (19/25)
F1 : 86.36%
False positives : 0 (out of 19 positive predictions)
False negatives : 6 (out of 111 negative predictions)


In [20]:
# export the results for debug
P_svm.to_csv('svm_debug1.csv')

True

In [21]:
# Now do Cross Validation on SVM with table I (H)
# Recall the cv for just the matcher
result = mg.cv_matcher_and_trigger(svm, [], table = H, 
                                   exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
                                  target_attr='gold')
result['cv_stats']

0%  100%
[#####] | ETA[sec]: 0.000 
Total time elapsed: 0.238 sec


,Metric,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,precision,5,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,recall,5,0.800000,0.800000,0.500000,0.875000,0.909091,0.776818
2,f1,5,0.888889,0.888889,0.666667,0.933333,0.952381,0.866032


# Training Iteration 2

In [ ]:
# After debugging, need to add 'telephone' related feature

In [ ]:
help(mg.get_feature_fn)

In [22]:
# Generate features
feat_table = mg.get_features_for_matching(A, B)

In [23]:
# Create features for phone in learning process
feat_phone1 = mg.get_feature_fn("exact_match(ltuple['telephone'], rtuple['phone'])", mg._match_t, mg._match_s)
# Add feature to feature table
mg.add_feature(feat_table, 'phone_phone_exm', feat_phone1)

True

In [24]:
feat_table

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source
0,id_id_exm,id,id,None,None,exact_match,<function id_id_exm at 0x123699848>,from magellan.feature.simfunctions import *\nf...
1,id_id_anm,id,id,None,None,abs_norm,<function id_id_anm at 0x1228b5e60>,from magellan.feature.simfunctions import *\nf...
2,id_id_lev,id,id,None,None,lev,<function id_id_lev at 0x1228b5ed8>,from magellan.feature.simfunctions import *\nf...
3,name_name_jac_qgm_3_qgm_3,name,name,qgm_3,qgm_3,jaccard,<function name_name_jac_qgm_3_qgm_3 at 0x1228b...,from magellan.feature.simfunctions import *\nf...
4,name_name_cos_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,cosine,<function name_name_cos_dlm_dc0_dlm_dc0 at 0x1...,from magellan.feature.simfunctions import *\nf...
5,name_name_jac_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,jaccard,<function name_name_jac_dlm_dc0_dlm_dc0 at 0x1...,from magellan.feature.simfunctions import *\nf...
6,name_name_mel,name,name,None,None,monge_elkan,<function name_name_mel at 0x1228b5578>,from magellan.feature.simfunctions import *\nf...
7,name_name_lev,name,name,None,None,lev,<function name_name_lev at 0x1228b5050>,from magellan.feature.simfunctions import *\nf...
8,name_name_nmw,name,name,None,None,needleman_wunsch,<function name_name_nmw at 0x1228b5758>,from magellan.feature.simfunctions import *\nf...
9,name_name_sw,name,name,None,None,smith_waterman,<function name_name_sw at 0x1228b50c8>,from magellan.feature.simfunctions import *\nf...


In [25]:
# select the original features first
feat1= feat_subset_iter1
# now select the new added features
feat2= feat_table[53:54]
# now append the the features sets
feat_subset_iter2= feat1.append(feat2)
feat_subset_iter2

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source
3,name_name_jac_qgm_3_qgm_3,name,name,qgm_3,qgm_3,jaccard,<function name_name_jac_qgm_3_qgm_3 at 0x12369...,from magellan.feature.simfunctions import *\nf...
4,name_name_cos_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,cosine,<function name_name_cos_dlm_dc0_dlm_dc0 at 0x1...,from magellan.feature.simfunctions import *\nf...
5,name_name_jac_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,jaccard,<function name_name_jac_dlm_dc0_dlm_dc0 at 0x1...,from magellan.feature.simfunctions import *\nf...
6,name_name_mel,name,name,None,None,monge_elkan,<function name_name_mel at 0x123691a28>,from magellan.feature.simfunctions import *\nf...
7,name_name_lev,name,name,None,None,lev,<function name_name_lev at 0x123691aa0>,from magellan.feature.simfunctions import *\nf...
8,name_name_nmw,name,name,None,None,needleman_wunsch,<function name_name_nmw at 0x123691b18>,from magellan.feature.simfunctions import *\nf...
9,name_name_sw,name,name,None,None,smith_waterman,<function name_name_sw at 0x123691b90>,from magellan.feature.simfunctions import *\nf...
10,name_name_swg,name,name,None,None,smith_waterman_gotoh,<function name_name_swg at 0x123691c08>,from magellan.feature.simfunctions import *\nf...
11,streetAddress_streetAddress_jac_qgm_3_qgm_3,streetAddress,streetAddress,qgm_3,qgm_3,jaccard,<function streetAddress_streetAddress_jac_qgm_...,from magellan.feature.simfunctions import *\nf...
12,streetAddress_streetAddress_cos_dlm_dc0_dlm_dc0,streetAddress,streetAddress,dlm_dc0,dlm_dc0,cosine,<function streetAddress_streetAddress_cos_dlm_...,from magellan.feature.simfunctions import *\nf...


In [26]:
# Get feature vectors
H = mg.extract_feature_vecs(I, feature_table=feat_subset_iter2, attrs_after='gold')
# impute H
H.fillna(0, inplace=True)
H.head(5)

,_id,ltable.id,rtable.id,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev,name_name_nmw,name_name_sw,...,state_state_jar,state_state_jwn,state_state_sdx,state_state_exm,state_state_jac_qgm_3_qgm_3,zipCode_zipCode_exm,zipCode_zipCode_anm,zipCode_zipCode_lev,phone_phone_exm,gold
0,0,6569,19993,0.454545,0.500000,0.333333,0.775000,0.650000,0.882353,0.750000,...,1,1,1,1,0,0,0.999834,0.428571,0,0
1,1,5883,15732,0.310345,0.408248,0.250000,0.725000,0.500000,0.711538,0.687500,...,1,1,1,1,0,0,0.999718,0.428571,0,0
2,2,7939,23163,0.652174,0.666667,0.500000,0.578947,0.863636,0.950000,0.973684,...,1,1,1,1,0,1,1.000000,0.714286,1,1
3,3,4812,12321,0.724138,0.750000,0.600000,0.520000,0.892857,0.961538,0.980000,...,1,1,1,1,0,1,1.000000,0.714286,1,1
4,4,9081,27265,0.250000,0.408248,0.250000,0.615385,0.476190,0.714286,0.692308,...,1,1,1,1,0,0,0.999853,0.428571,0,0


In [27]:
# select the best ML matcher using CV
result_p = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='precision', random_state=0)
result_r = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='recall', random_state=0)
result_f1 = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='f1', random_state=0)
# print results
print "Precision: "
print result_p['cv_stats']
print "Recall: "
print result_r['cv_stats']
print "F1: "
print result_f1['cv_stats']

Precision: 
                       Name  \
0              DecisionTree   
1                        RF   
2  SVM_65898401806274538441   
3                        NB   
4                    LogReg   
5                    LinReg   

                                             Matcher  Num folds    Fold 1  \
0  <magellan.matcher.dtmatcher.DTMatcher object a...          5  1.000000   
1  <magellan.matcher.rfmatcher.RFMatcher object a...          5  1.000000   
2  <magellan.matcher.svmmatcher.SVMMatcher object...          5  1.000000   
3  <magellan.matcher.nbmatcher.NBMatcher object a...          5  1.000000   
4  <magellan.matcher.logregmatcher.LogRegMatcher ...          5  1.000000   
5  <magellan.matcher.linregmatcher.LinRegMatcher ...          5  0.923077   

   Fold 2  Fold 3    Fold 4    Fold 5  Mean score  
0   1.000   1.000  1.000000  0.857143    0.971429  
1   1.000   1.000  1.000000  1.000000    1.000000  
2   1.000   1.000  1.000000  0.857143    0.971429  
3   1.000   0.875  0.7

In [ ]:
# Now choose Random Forest RF

In [28]:
# Split feature vectors into U and V
UV = mg.train_test_split(H, train_proportion=0.5, random_state=0)
U = UV['train']
V = UV['test']

In [ ]:
# do proxy debugging using DT and RF

In [29]:
# debug DT results
mg.vis_debug_dt(dt, U, V, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],target_attr='gold')
# precision: 100.0% (23/23)
# recall: 92.0% (23/25)
# F1: 95.83%

In [30]:
# debug RF results
mg.vis_debug_rf(rf, U, V, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],target_attr='gold')
# precision: 96.0% (24/25)
# recall: 96.0% (24/25)
# F1: 96.0%

In [31]:
# Now do Cross Validation on Random Forest with table I (H)
# Recall the cv for just the matcher
result = mg.cv_matcher_and_trigger(rf, [], table = H, 
                                   exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
                                  target_attr='gold')
result['cv_stats']

0%  100%
[#####] | ETA[sec]: 0.000 
Total time elapsed: 0.248 sec


,Metric,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,precision,5,1,1,1,1,0.846154,0.969231
1,recall,5,1,1,1,1,0.916667,0.983333
2,f1,5,1,1,1,1,0.880000,0.976000


# Training Iteration 3

In [ ]:
# Found incorrect label
# re-label data

In [32]:
# re-read I and J
I= mg.read_csv('I_relabel.csv', ltable=A, rtable=B)
J= mg.read_csv('J_relabel.csv', ltable=A, rtable=B)
# check the number of tuples in I and J
(len(I), len(J))

(260, 140)

In [33]:
# Select 'name', 'streetAddress', 'city', 'state', 'zipCode' related features
feat_table = mg.get_features_for_matching(A, B)
feat_subset_iter3 = feat_table[3:36]

In [34]:
# Get feature vectors
H = mg.extract_feature_vecs(I, feature_table=feat_subset_iter3, attrs_after='gold')
# impute H
H.fillna(0, inplace=True)
H.head()

,_id,ltable.id,rtable.id,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev,name_name_nmw,name_name_sw,...,state_state_lev,state_state_jar,state_state_jwn,state_state_sdx,state_state_exm,state_state_jac_qgm_3_qgm_3,zipCode_zipCode_exm,zipCode_zipCode_anm,zipCode_zipCode_lev,gold
0,0,6569,19993,0.454545,0.500000,0.333333,0.775000,0.650000,0.882353,0.750000,...,1,1,1,1,1,0,0,0.999834,0.428571,0
1,1,5883,15732,0.310345,0.408248,0.250000,0.725000,0.500000,0.711538,0.687500,...,1,1,1,1,1,0,0,0.999718,0.428571,0
2,2,7939,23163,0.652174,0.666667,0.500000,0.578947,0.863636,0.950000,0.973684,...,1,1,1,1,1,0,1,1.000000,0.714286,1
3,3,4812,12321,0.724138,0.750000,0.600000,0.520000,0.892857,0.961538,0.980000,...,1,1,1,1,1,0,1,1.000000,0.714286,1
4,4,9081,27265,0.250000,0.408248,0.250000,0.615385,0.476190,0.714286,0.692308,...,1,1,1,1,1,0,0,0.999853,0.428571,0


In [35]:
# select the best ML matcher using CV
result_p = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='precision', random_state=0)
result_r = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='recall', random_state=0)
result_f1 = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='f1', random_state=0)
# print results
print "Precision: "
print result_p['cv_stats']
print "Recall: "
print result_r['cv_stats']
print "F1: "
print result_f1['cv_stats']

Precision: 
                       Name  \
0              DecisionTree   
1                        RF   
2  SVM_65898401806274538441   
3                        NB   
4                    LogReg   
5                    LinReg   

                                             Matcher  Num folds    Fold 1  \
0  <magellan.matcher.dtmatcher.DTMatcher object a...          5  0.916667   
1  <magellan.matcher.rfmatcher.RFMatcher object a...          5  0.916667   
2  <magellan.matcher.svmmatcher.SVMMatcher object...          5  1.000000   
3  <magellan.matcher.nbmatcher.NBMatcher object a...          5  0.705882   
4  <magellan.matcher.logregmatcher.LogRegMatcher ...          5  1.000000   
5  <magellan.matcher.linregmatcher.LinRegMatcher ...          5  1.000000   

     Fold 2    Fold 3    Fold 4    Fold 5  Mean score  
0  0.857143  1.000000  1.000000  0.750000    0.904762  
1  1.000000  1.000000  1.000000  1.000000    0.983333  
2  1.000000  1.000000  1.000000  1.000000    1.000000  
3  0.8

In [ ]:
# SVM has the highest precision

In [39]:
# Split feature vectors to train and test
UV = mg.train_test_split(H, train_proportion=0.5, random_state=0)
U = UV['train']
V = UV['test']

In [40]:
# debug DT results
mg.vis_debug_dt(dt, U, V, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')
# precision: 91.67% (22/24)
# recall: 88.0% (22/25)
# F1: 89.8%

In [41]:
# debug RF results
mg.vis_debug_rf(rf, U, V, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')
# precision: 92.0% (23/25)
# recall: 92.0% (23/25)
# F1: 92.0%

In [42]:
# check SVM results
svm.fit(table=U, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')
# Predict V using SVM
P_svm3 = svm.predict(table=V, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)
# Evaluate the predictions
eval_result = mg.eval_matches(P_svm3, 'gold', 'predicted')
mg.print_eval_summary(eval_result)
# Precision : 100.0% (19/19)
# Recall : 76.0% (19/25)
# F1 : 86.36%

Precision : 100.0% (19/19)
Recall : 76.0% (19/25)
F1 : 86.36%
False positives : 0 (out of 19 positive predictions)
False negatives : 6 (out of 111 negative predictions)


In [43]:
# Now do Cross Validation on SVM with table I (H)
# Recall the cv for just the matcher
result = mg.cv_matcher_and_trigger(svm, [], table = H, 
                                   exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
                                  target_attr='gold')
result['cv_stats']

0%  100%
[#####] | ETA[sec]: 0.000 
Total time elapsed: 0.211 sec


,Metric,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,precision,5,1.000000,0.875,1.000000,1.000000,1.000000,0.975000
1,recall,5,0.888889,0.875,0.700000,0.461538,0.923077,0.769701
2,f1,5,0.941176,0.875,0.823529,0.631579,0.960000,0.846257


# Training Iteration 4

In [ ]:
# now add 'telephone' related features to the re-label data

In [44]:
# Generate features
feat_table = mg.get_features_for_matching(A, B)

In [45]:
# Create features for phone in learning process
feat_phone1 = mg.get_feature_fn("exact_match(ltuple['telephone'], rtuple['phone'])", mg._match_t, mg._match_s)
# Add feature to feature table
mg.add_feature(feat_table, 'phone_phone_exm', feat_phone1)

True

In [46]:
# select the original features first
feat1= feat_subset_iter3
# now select the new added features
feat2= feat_table[53:54]

In [47]:
# now append the the features sets
feat_subset_iter4= feat1.append(feat2)

In [48]:
# Get feature vectors
H = mg.extract_feature_vecs(I, feature_table=feat_subset_iter4, attrs_after='gold')
# impute H
H.fillna(0, inplace=True)
H.head(5)

,_id,ltable.id,rtable.id,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev,name_name_nmw,name_name_sw,...,state_state_jar,state_state_jwn,state_state_sdx,state_state_exm,state_state_jac_qgm_3_qgm_3,zipCode_zipCode_exm,zipCode_zipCode_anm,zipCode_zipCode_lev,phone_phone_exm,gold
0,0,6569,19993,0.454545,0.500000,0.333333,0.775000,0.650000,0.882353,0.750000,...,1,1,1,1,0,0,0.999834,0.428571,0,0
1,1,5883,15732,0.310345,0.408248,0.250000,0.725000,0.500000,0.711538,0.687500,...,1,1,1,1,0,0,0.999718,0.428571,0,0
2,2,7939,23163,0.652174,0.666667,0.500000,0.578947,0.863636,0.950000,0.973684,...,1,1,1,1,0,1,1.000000,0.714286,1,1
3,3,4812,12321,0.724138,0.750000,0.600000,0.520000,0.892857,0.961538,0.980000,...,1,1,1,1,0,1,1.000000,0.714286,1,1
4,4,9081,27265,0.250000,0.408248,0.250000,0.615385,0.476190,0.714286,0.692308,...,1,1,1,1,0,0,0.999853,0.428571,0,0


In [49]:
# select the best ML matcher using CV
result_p = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='precision', random_state=0)
result_r = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='recall', random_state=0)
result_f1 = mg.select_matcher([dt, rf, svm, nb, lg, ln], table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='f1', random_state=0)
# print results
print "Precision: "
print result_p['cv_stats']
print "Recall: "
print result_r['cv_stats']
print "F1: "
print result_f1['cv_stats']

Precision: 
                       Name  \
0              DecisionTree   
1                        RF   
2  SVM_65898401806274538441   
3                        NB   
4                    LogReg   
5                    LinReg   

                                             Matcher  Num folds    Fold 1  \
0  <magellan.matcher.dtmatcher.DTMatcher object a...          5  0.923077   
1  <magellan.matcher.rfmatcher.RFMatcher object a...          5  1.000000   
2  <magellan.matcher.svmmatcher.SVMMatcher object...          5  1.000000   
3  <magellan.matcher.nbmatcher.NBMatcher object a...          5  1.000000   
4  <magellan.matcher.logregmatcher.LogRegMatcher ...          5  1.000000   
5  <magellan.matcher.linregmatcher.LinRegMatcher ...          5  0.923077   

   Fold 2    Fold 3    Fold 4    Fold 5  Mean score  
0   1.000  1.000000  1.000000  1.000000    0.984615  
1   1.000  1.000000  1.000000  1.000000    1.000000  
2   1.000  1.000000  1.000000  1.000000    1.000000  
3   1.000  0.7

In [ ]:
# select Random Forest RF

In [50]:
# Split feature vectors into U and V
UV = mg.train_test_split(H, train_proportion=0.5, random_state=0)
U = UV['train']
V = UV['test']

In [51]:
# debug RF results
mg.vis_debug_rf(rf, U, V, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')
# precision: 92.59% (25/27)
# recall: 100.0% (25/25)
# F1: 96.15%

In [52]:
# debug DT results
mg.vis_debug_dt(dt, U, V, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')
# precision: 96.0% (24/25)
# recall: 96.0% (24/25)
# F1: 96.0%

In [53]:
# check SVM results
svm.fit(table=U, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')
# Predict V using SVM
P = svm.predict(table=V, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)
# Evaluate the predictions
eval_result = mg.eval_matches(P, 'gold', 'predicted')
mg.print_eval_summary(eval_result)
# Precision : 100.0% (24/24)
# Recall : 96.0% (24/25)
# F1 : 97.96%

Precision : 100.0% (24/24)
Recall : 96.0% (24/25)
F1 : 97.96%
False positives : 0 (out of 24 positive predictions)
False negatives : 1 (out of 106 negative predictions)


In [54]:
# Now do Cross Validation on Random Forest with table I (H)
# Recall the cv for just the matcher
result = mg.cv_matcher_and_trigger(rf, [], table = H, 
                                   exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
                                  target_attr='gold')
result['cv_stats']

0%  100%
[#####] | ETA[sec]: 0.000 
Total time elapsed: 0.255 sec


,Metric,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,precision,5,0.909091,1,1,1,1,0.981818
1,recall,5,1.000000,1,1,1,1,1.000000
2,f1,5,0.952381,1,1,1,1,0.990476


# Testing on current ML matchers

In [55]:
# Extract feature vectors
M = mg.extract_feature_vecs(J, feature_table=feat_subset_iter4, attrs_after='gold')
# impute M
M.fillna(0, inplace=True)
M.head(5)

,_id,ltable.id,rtable.id,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev,name_name_nmw,name_name_sw,...,state_state_jar,state_state_jwn,state_state_sdx,state_state_exm,state_state_jac_qgm_3_qgm_3,zipCode_zipCode_exm,zipCode_zipCode_anm,zipCode_zipCode_lev,phone_phone_exm,gold
0,0,3509,9278,0.137931,0.316228,0.166667,0.428571,0.333333,0.520833,0.428571,...,1,1,1,1,0,1,1.000000,0.714286,1,1
1,1,4324,14561,0.428571,0.408248,0.250000,0.647059,0.647059,0.823529,0.647059,...,1,1,1,1,0,0,0.999530,0.428571,0,0
2,2,9452,28718,0.250000,0.250000,0.142857,0.545455,0.500000,0.703125,0.568182,...,1,1,1,1,0,1,1.000000,0.714286,0,0
3,3,8446,24035,0.523810,0.816497,0.666667,1.000000,0.565217,0.782609,1.000000,...,1,1,1,1,0,1,1.000000,0.714286,1,1
4,4,8729,24364,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1,1,1,1,0,0,0.999973,0.571429,1,1


In [56]:
# check DT results
dt.fit(table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],target_attr='gold')
# Predict V using DT
N_dt = dt.predict(table=M, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)
# Evaluate the predictions
eval_result = mg.eval_matches(N_dt, 'gold', 'predicted')
mg.print_eval_summary(eval_result)
# Precision : 91.67% (33/36)
# Recall : 94.29% (33/35)
# F1 : 92.96%

Precision : 91.67% (33/36)
Recall : 94.29% (33/35)
F1 : 92.96%
False positives : 3 (out of 36 positive predictions)
False negatives : 2 (out of 104 negative predictions)


In [57]:
# check RF results
rf.fit(table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],target_attr='gold')
# Predict V using RF
N_rf = rf.predict(table=M, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)
# Evaluate the predictions
eval_result = mg.eval_matches(N_rf, 'gold', 'predicted')
mg.print_eval_summary(eval_result)
# Precision : 94.44% (34/36)
# Recall : 97.14% (34/35)
# F1 : 95.77%

Precision : 94.44% (34/36)
Recall : 97.14% (34/35)
F1 : 95.77%
False positives : 2 (out of 36 positive predictions)
False negatives : 1 (out of 104 negative predictions)


In [58]:
# check SVM results
svm.fit(table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],target_attr='gold')
# Predict V using SVM
N_svm = svm.predict(table=M, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)
# Evaluate the predictions
eval_result = mg.eval_matches(N_svm, 'gold', 'predicted')
mg.print_eval_summary(eval_result)
# Precision : 94.29% (33/35)
# Recall : 94.29% (33/35)
# F1 : 94.29%

Precision : 94.29% (33/35)
Recall : 94.29% (33/35)
F1 : 94.29%
False positives : 2 (out of 35 positive predictions)
False negatives : 2 (out of 105 negative predictions)


In [59]:
# check NB results
nb.fit(table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],target_attr='gold')
# Predict V using NB
N_nb = nb.predict(table=M, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)
# Evaluate the predictions
eval_result = mg.eval_matches(N_nb, 'gold', 'predicted')
mg.print_eval_summary(eval_result)
# Precision : 85.0% (34/40)
# Recall : 97.14% (34/35)
# F1 : 90.67%

Precision : 85.0% (34/40)
Recall : 97.14% (34/35)
F1 : 90.67%
False positives : 6 (out of 40 positive predictions)
False negatives : 1 (out of 100 negative predictions)


In [60]:
# check LG results
lg.fit(table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],target_attr='gold')
# Predict V using SVM
N_lg = lg.predict(table=M, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)
# Evaluate the predictions
eval_result = mg.eval_matches(N_lg, 'gold', 'predicted')
mg.print_eval_summary(eval_result)
# Precision : 94.44% (34/36)
# Recall : 97.14% (34/35)
# F1 : 95.77%

Precision : 94.44% (34/36)
Recall : 97.14% (34/35)
F1 : 95.77%
False positives : 2 (out of 36 positive predictions)
False negatives : 1 (out of 104 negative predictions)


In [61]:
# check LN results
ln.fit(table=H, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],target_attr='gold')
# Predict V using LN
N_ln = ln.predict(table=M, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)
# Evaluate the predictions
eval_result = mg.eval_matches(N_ln, 'gold', 'predicted')
mg.print_eval_summary(eval_result)
# Precision : 91.67% (33/36)
# Recall : 94.29% (33/35)
# F1 : 92.96%

Precision : 91.67% (33/36)
Recall : 94.29% (33/35)
F1 : 92.96%
False positives : 3 (out of 36 positive predictions)
False negatives : 2 (out of 104 negative predictions)


In [62]:
# output final matches
N_rf.to_csv('final_matches_rf.csv')

True